In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
base_dir = "C://DATA//train//49_kaggle//KaggleFun//ChestXRay//DATA//chest_xray"

In [2]:
from torchvision import transforms
image_transforms = {
    # Train uses data augmentation
    'train':
    transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),  
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  
    ]),
    # Validation does not use augmentation
    'test':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [3]:
from torchvision.datasets import ImageFolder

train_ds = ImageFolder(root=os.path.join(base_dir,"train"), transform=image_transforms['train'])
test_ds = ImageFolder(root=os.path.join(base_dir,"test"), transform=image_transforms['test'])
print(len(train_ds), len(test_ds))
class_names = train_ds.classes
print(class_names)

5216 640
['NORMAL', 'PNEUMONIA']


In [4]:
from torch.utils.data.dataloader import DataLoader

batch_size = 64

train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=3, pin_memory=True)
test_dl = DataLoader(test_ds, batch_size, num_workers=3, pin_memory=True)

device = torch.device("cpu") if not torch.cuda.is_available() else torch.device("cuda")

In [5]:
images, labels = iter(train_dl).next()
images.shape

torch.Size([64, 3, 224, 224])

In [16]:
import torch.nn as nn

class Net(nn.Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = nn.Sequential(
            # Defining a 2D convolution layer
            nn.Conv2d(3, 4, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(4, 8, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(8),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=3),
        )

        self.linear_layers = nn.Sequential(
            nn.Linear(8 * 37 * 37, 2)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [22]:
import torch.optim as optim

model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)
EPOCHS = 15

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(3, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=3, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=10952, out_features=2, bias=True)
  )
)


In [23]:
for epoch in range(EPOCHS):  # loop over the dataset multiple times
    
    model.train()
    running_loss = 0.0
    total, correct = 0, 0
    for i, data in enumerate(train_dl, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    print("Epoch: {}  Train Acc: {:.2f}".format(epoch, correct/total*100))
    
    model.eval()
    total, correct = 0, 0
    for i, data in enumerate(test_dl, 0):
        inputs, labels = data
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print("Epoch: {}  Test Acc: {:.2f}".format(epoch, correct/total*100))
print('Finished Training')

Epoch: 0  Train Acc: 87.44
Epoch: 0  Test Acc: 73.12
Epoch: 1  Train Acc: 92.27
Epoch: 1  Test Acc: 68.75
Epoch: 2  Train Acc: 92.94
Epoch: 2  Test Acc: 74.53
Epoch: 3  Train Acc: 93.79
Epoch: 3  Test Acc: 72.81
Epoch: 4  Train Acc: 94.33
Epoch: 4  Test Acc: 74.22
Epoch: 5  Train Acc: 94.25
Epoch: 5  Test Acc: 74.69
Epoch: 6  Train Acc: 94.23
Epoch: 6  Test Acc: 70.31
Epoch: 7  Train Acc: 94.29
Epoch: 7  Test Acc: 74.38
Epoch: 8  Train Acc: 94.31
Epoch: 8  Test Acc: 73.59
Epoch: 9  Train Acc: 94.56
Epoch: 9  Test Acc: 73.59
Epoch: 10  Train Acc: 93.88
Epoch: 10  Test Acc: 70.00
Epoch: 11  Train Acc: 94.57
Epoch: 11  Test Acc: 69.69
Epoch: 12  Train Acc: 94.69
Epoch: 12  Test Acc: 73.91
Epoch: 13  Train Acc: 95.02
Epoch: 13  Test Acc: 71.56
Epoch: 14  Train Acc: 95.05
Epoch: 14  Test Acc: 74.53
Finished Training


There's no dropout so it probably overfit